PREDICTIVE MODEL

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
hist = pd.read_csv('historical_data.csv')
fg = pd.read_csv('fear_greed_index.csv')

/tmp/ipython-input-3129335900.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  hist = pd.read_csv('historical_data.csv')


In [5]:
hist['time'] = pd.to_datetime(hist['Timestamp IST'], errors='coerce')
fg['date'] = pd.to_datetime(fg['date'], errors='coerce')
hist['date'] = hist['time'].dt.floor('D')

fg['sentiment_score'] = fg['classification'].str.lower().map({
    'fear': 0,
    'extreme fear': 0,
    'greed': 1,
    'extreme greed': 1,
    'neutral': 0.5
})

In [8]:
daily_trades = hist.groupby('date').agg(
    trades_count=('Account', 'count'),
    total_volume=('Size USD', 'sum'),
    avg_leverage=('Start Position', 'mean'),
    net_pnl=('Closed PnL', 'sum')
).reset_index()

daily = daily_trades.merge(fg[['date', 'sentiment_score']], on='date', how='left')

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

daily['profitable'] = (daily['net_pnl'] > 0).astype(int)
X = daily[['sentiment_score', 'trades_count', 'total_volume', 'avg_leverage']]
y = daily['profitable']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Model Accuracy:", clf.score(X_test, y_test))
print(classification_report(y_test, y_pred))


Model Accuracy: 0.8157894736842105
              precision    recall  f1-score   support

           0       0.50      0.29      0.36         7
           1       0.85      0.94      0.89        31

    accuracy                           0.82        38
   macro avg       0.68      0.61      0.63        38
weighted avg       0.79      0.82      0.79        38

